In [ ]:
!pip install requests_html

In [2]:
from requests_html import HTMLSession
import pandas as pd
import requests

In [3]:
# Parse particular news for tickers' mentions,takes in an url of an article to pass and an instance of a parsing session from page parser
def url_parse(parse_url, parse_session):
  parse_request = parse_session.get(parse_url)
  content = parse_request.html.find('section.release-body')

  try:
    for item in content:
      parse_ticker = item.find('a.ticket-symbol', first=True).text
  except AttributeError:
    parse_ticker = None
  try:
    return parse_ticker
  except UnboundLocalError:
    return None # Return non if no tickers found

In [8]:
# Function to parse a parse a particular page for all the news to later parse them for tickers. Takes 2 parameters: a number of pages and initial dataset of already saved news.
def page_parse(x, page_session, data=[]):
    page_url = f'https://www.prnewswire.com/news-releases/english-releases/?page={x}&pagesize=100'
    page_request = page_session.get(page_url)
    content = page_request.html.find('div.row.arabiclistingcards')
    for item in content:
        date = item.find('h3', first=True).text.split('ET')[-2]
        title = item.find('h3', first=True).text.split('ET')[-1]
        article_url = 'https://www.prnewswire.com' + item.find('a.newsreleaseconsolidatelink', first=True).attrs['href']
        ticker = url_parse(article_url, page_session)
        try:
          dic = {
              'Date': pd.to_datetime(date),
              'Title': title,
              'Ticker': ticker
          }
        except Exception:
          pass
        data.append(dic)
    return data

In [9]:
# Main scanner, takes a number of pages to parse with 10 being a default. 50 pages cover a week of news and 100 pages cover 2 weeks of data.
def run_scanner(pages=10):
  session = HTMLSession()
  data = []

  for x in range(1, pages+1):
      page_parse(x, session, data)

  df = pd.DataFrame(data)
  df.dropna(subset=['Ticker'], inplace=True)
  df.set_index('Date', inplace=True)
  return df

In [10]:
df = run_scanner(10)

In [11]:
df

,Title,Ticker
Date,,
2021-11-06 13:47:00,Edwards Announces Six-month Data From Transca...,EW
2021-11-06 13:34:00,Medtronic announces early data for the Intrep...,MDT
2021-11-06 13:22:00,"Did You Purchase Stock Between March 22, 2021...",BIDU
2021-11-06 13:05:00,SHAREHOLDER INVESTIGATION: Halper Sadeh LLP I...,DWSN
2021-11-06 12:53:00,SHAREHOLDER INVESTIGATION: Halper Sadeh LLP I...,PAE
...,...,...
2021-10-25 10:01:00,Tri Pointe Homes Wins Big at the Great Americ...,TPH
2021-10-25 10:00:00,Comerica Bank earns Best in Enterprise Resili...,CMA
2021-10-25 10:00:00,"Peloton Releases Inaugural Environmental, Soc...",PTON


In [12]:
df.to_csv('Oct25Nov6.csv') # Save dataframe as a csv for further analysis